In [15]:
!pip install paddlepaddle paddleocr

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.2 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
kraken 5.3.0 requires numpy~=2.0.0, but you have numpy 1.26.4 which is incompatible.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.2 which is incompatible.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
tensorflow-intel 2.16.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.



  Using cached protobuf-3.20.2-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.2-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3


In [11]:
#Text extracted Using Paddle ocr after image Processing and compared to Google cloud vision api that has won in 
#deleivering more accurate text(numbers) in the image

In [13]:
import os
import cv2
import numpy as np
from paddleocr import PaddleOCR
import re
import csv
from google.cloud import vision

# Ask user if they have a Google Cloud account
use_google_vision = input("Do you have a Google Cloud account set up for Vision API? (yes/no): ").strip().lower()
use_google_vision = use_google_vision == "yes"

# Path to the folder containing images
folder_path = r"C:\Users\HP\Downloads\DATA-20250127T075136Z-001\DATA"

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# If Google Vision is enabled, set credentials
if use_google_vision:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\HP\Downloads\........"
    client = vision.ImageAnnotatorClient()

# Output CSV file path
output_csv_path = r"C:\Users\HP\Documents\extracted_text_numbers_combined.csv"

# Prepare the CSV file with headers
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Name", "Extracted Numbers"])

image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# ===================== PaddleOCR Processing =====================
for idx, image_name in enumerate(image_files):
    image_path = os.path.join(folder_path, image_name)

    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Could not load image: {image_name}")
        continue

    # Apply preprocessing
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    denoised = cv2.GaussianBlur(binary, (5, 5), 0)
    image_rgb = cv2.cvtColor(denoised, cv2.COLOR_GRAY2RGB)

    # Run OCR using PaddleOCR
    result = ocr.ocr(image_rgb, cls=True)
    text = "".join([line[1][0] for line in result[0]])
    numbers_paddle = re.findall(r'\d+\.?\d*', text)
    numbers_paddle_string = ", ".join(numbers_paddle)

    # Generate new image name
    new_image_name = f"map_{str(idx + 1).zfill(3)}.jpeg"

    # Write PaddleOCR data to CSV
    with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([new_image_name, numbers_paddle_string])

    print(f"Processed: {new_image_name} (PaddleOCR)")

# ===================== Google Cloud Vision Processing (If Enabled) =====================
if use_google_vision:
    for idx, image_name in enumerate(image_files):
        image_path = os.path.join(folder_path, image_name)
        
        with open(image_path, 'rb') as image_file:
            content = image_file.read()
        
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        annotations = response.text_annotations
        full_text = annotations[0].description if annotations else ""
        numbers_vision = re.findall(r'\d+\.?\d*', full_text)
        numbers_vision_string = ", ".join(numbers_vision)
        
        new_image_name_vision = f"map_vision_{str(idx + 1).zfill(3)}.jpeg"
        
        with open(output_csv_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([new_image_name_vision, numbers_vision_string])
        
        print(f"Processed: {new_image_name_vision} (Google Cloud Vision)")

print(f"Processing complete. CSV saved at: {output_csv_path}")


Do you have a Google Cloud account set up for Vision API? (yes/no):  no


[2025/01/29 14:17:53] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\HP/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\HP/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max